In [ ]:
import pandas as pd
from textblob import TextBlob
import yfinance as yf
import os
import scraper
import numpy as np
import csv

Estos son los tickers de las empresas más relevantes para la tecnología de acuerdo a Yahoo! Finance.

In [ ]:
TICKERS = [
    'AAPL', 'MSFT', 'GOOG', 'AMZN', 'NVDA', 'META',
    'TSLA', 'BABA', 'CRM', 'AMD', 'INTC', 'PYPL',
    'ATVI', 'EA', 'TTD', 'ZG', 'MTCH', 'YELP',
]
initial_results_path = os.path.join("./results/")
stock_hist_path = os.path.join(initial_results_path, "stock_hist.csv")
news_path = os.path.join("./results/", "output_des.csv")
final_results_path = os.path.join("./final_results/")

Se necesita que el servidor de noticias de RSShub para poder sacar las noticias. Es mejor correrlo por separado ya que puede imprimir un log muy grande.

In [ ]:
#scraper.get_news(initial_results_path)

Sacamos el stock histórico de todos los tickers que se establecieron antes.

In [ ]:
stock_columns = ['Ticker','Date','Open','High','Low','Close','Volume','Dividends','Stock Splits']
tickers = yf.Tickers(' '.join(TICKERS))
stocks = pd.DataFrame(columns=stock_columns)
for ticker in TICKERS:
    data = tickers.tickers[ticker].history(period="max")
    stock = pd.DataFrame(data)
    stock = stock.reset_index()
    stock["Ticker"] = ticker
    stocks = pd.concat([stocks, stock], sort=False)

stocks.reset_index()
stocks.to_csv(stock_hist_path, index=False)

Le agregamos el análisis de sentimientos a cada noticia.

In [63]:
news = pd.read_csv(news_path)
news.tail()

,ticker,title,description,pub_date,link,author
1795,YELP,Companies announce abortion benefits as trigge...,Companies announce abortion benefits as trigge...,"Mon, 27 Jun 2022 14:46:00 GMT",https://finance.yahoo.com/video/companies-anno...,Yahoo Finance Video
1796,YELP,Yelp closing D.C. office in commitment to remo...,Yelp closing D.C. office in commitment to remo...,"Mon, 27 Jun 2022 16:49:00 GMT",https://www.bizjournals.com/washington/news/20...,American City Business Journals
1797,YELP,"Yelp (YELP) Embraces Remote Work Culture, Clos...","Yelp (YELP) Embraces Remote Work Culture, Clos...","Mon, 27 Jun 2022 14:06:00 GMT",https://finance.yahoo.com/news/yelp-yelp-embra...,Zacks
1798,YELP,Yelp closes 3 U.S. offices as it shifts to rem...,Yelp closes 3 U.S. offices as it shifts to rem...,"Fri, 24 Jun 2022 21:09:00 GMT",https://finance.yahoo.com/video/yelp-closes-3-...,Yahoo Finance Video
1799,YELP,Here's how Silicon Valley's companies are resp...,Here's how Silicon Valley's companies are resp...,"Fri, 24 Jun 2022 23:24:00 GMT",https://www.bizjournals.com/sanjose/news/2022/...,American City Business Journals


In [64]:
def get_sentiment(row):
    sentiments_title = TextBlob(str(row['title'])).sentiment
    sentiments_description = TextBlob(str(row['description'])).sentiment
    results = list(sentiments_title)
    results.extend(list(sentiments_description))
    return results

news[[
    'polarity_title','subjectivity_title',
    'polarity_description','subjectivity_description'
]] = news.apply(lambda row: pd.Series(get_sentiment(row)), axis=1)

news.tail()

,ticker,title,description,pub_date,link,author,polarity_title,subjectivity_title,polarity_description,subjectivity_description
1795,YELP,Companies announce abortion benefits as trigge...,Companies announce abortion benefits as trigge...,"Mon, 27 Jun 2022 14:46:00 GMT",https://finance.yahoo.com/video/companies-anno...,Yahoo Finance Video,0.0,0.0,0.0,0.0
1796,YELP,Yelp closing D.C. office in commitment to remo...,Yelp closing D.C. office in commitment to remo...,"Mon, 27 Jun 2022 16:49:00 GMT",https://www.bizjournals.com/washington/news/20...,American City Business Journals,-0.1,0.2,-0.1,0.2
1797,YELP,"Yelp (YELP) Embraces Remote Work Culture, Clos...","Yelp (YELP) Embraces Remote Work Culture, Clos...","Mon, 27 Jun 2022 14:06:00 GMT",https://finance.yahoo.com/news/yelp-yelp-embra...,Zacks,-0.1,0.2,-0.1,0.2
1798,YELP,Yelp closes 3 U.S. offices as it shifts to rem...,Yelp closes 3 U.S. offices as it shifts to rem...,"Fri, 24 Jun 2022 21:09:00 GMT",https://finance.yahoo.com/video/yelp-closes-3-...,Yahoo Finance Video,-0.1,0.2,-0.1,0.2
1799,YELP,Here's how Silicon Valley's companies are resp...,Here's how Silicon Valley's companies are resp...,"Fri, 24 Jun 2022 23:24:00 GMT",https://www.bizjournals.com/sanjose/news/2022/...,American City Business Journals,0.0,0.0,0.0,0.0


Ahora le agregamos los valores de stock a cada día relevante dependiendo que día afectó la notica. Considerando la hora en que se abre los stocks en UTC.

In [65]:
nyse_holidays = [
    "2023-01-02",
    "2023-01-16",
    "2023-02-20",
    "2023-04-14",
    "2023-05-29",
    "2023-07-04",
    "2023-09-04",
    "2023-11-23",
    "2023-12-25"
]
EXAMPLE = "AAPL"
STOCK_OPENING_HOUR = 10 #UTC
STOCK_OPENING_MIN = 30 #UTC
STOCK_CLOSE_HOUR = 20 #UTC
STOCK_CLOSE_MIN = 0 #UTC

En este caso, es mucho mejor usar la fecha como index ya que hace más fácil obtener la fecha específica de un stock.

In [66]:
stock_hist_path = os.path.join(initial_results_path, "stock_hist.csv")
stocks = pd.read_csv(stock_hist_path)
news["date"] = pd.to_datetime(news['pub_date'],errors='coerce',utc=True).dt.tz_localize(None)
stocks['Date'] = pd.to_datetime(stocks["Date"], errors="coerce",utc=True).dt.tz_localize(None)
stocks.set_index('Date', inplace=True)

In [67]:
def get_affected_day(date):
    close_hour = date.replace(hour=STOCK_CLOSE_HOUR, minute=STOCK_CLOSE_MIN)
    if (date > close_hour) or (date.strftime("%Y-%m-%d") in nyse_holidays):
        date = date.replace(hour=STOCK_OPENING_HOUR, minute=STOCK_OPENING_MIN)
        date = date + pd.Timedelta(days=1)
    return date

news = news.dropna(axis=0)
news["affected_day"] = news["date"].apply(get_affected_day)
news.tail()

,ticker,title,description,pub_date,link,author,polarity_title,subjectivity_title,polarity_description,subjectivity_description,date,affected_day
1795,YELP,Companies announce abortion benefits as trigge...,Companies announce abortion benefits as trigge...,"Mon, 27 Jun 2022 14:46:00 GMT",https://finance.yahoo.com/video/companies-anno...,Yahoo Finance Video,0.0,0.0,0.0,0.0,2022-06-27 14:46:00,2022-06-27 14:46:00
1796,YELP,Yelp closing D.C. office in commitment to remo...,Yelp closing D.C. office in commitment to remo...,"Mon, 27 Jun 2022 16:49:00 GMT",https://www.bizjournals.com/washington/news/20...,American City Business Journals,-0.1,0.2,-0.1,0.2,2022-06-27 16:49:00,2022-06-27 16:49:00
1797,YELP,"Yelp (YELP) Embraces Remote Work Culture, Clos...","Yelp (YELP) Embraces Remote Work Culture, Clos...","Mon, 27 Jun 2022 14:06:00 GMT",https://finance.yahoo.com/news/yelp-yelp-embra...,Zacks,-0.1,0.2,-0.1,0.2,2022-06-27 14:06:00,2022-06-27 14:06:00
1798,YELP,Yelp closes 3 U.S. offices as it shifts to rem...,Yelp closes 3 U.S. offices as it shifts to rem...,"Fri, 24 Jun 2022 21:09:00 GMT",https://finance.yahoo.com/video/yelp-closes-3-...,Yahoo Finance Video,-0.1,0.2,-0.1,0.2,2022-06-24 21:09:00,2022-06-25 10:30:00
1799,YELP,Here's how Silicon Valley's companies are resp...,Here's how Silicon Valley's companies are resp...,"Fri, 24 Jun 2022 23:24:00 GMT",https://www.bizjournals.com/sanjose/news/2022/...,American City Business Journals,0.0,0.0,0.0,0.0,2022-06-24 23:24:00,2022-06-25 10:30:00


En la función siguiente se agregan los aperturas y cerraduras de los stocks dependiendo de las fechas.

In [68]:
def add_stock_results(row):
    date = row["affected_day"].strftime("%Y-%m-%d")
    stock_values = stocks[
        (stocks['Ticker'] == row['ticker'])
    ]
    stock_values = stock_values.loc[date, ["Open","Close"]]
    if len(stock_values.values) != 0:
        stock_values = stock_values.values[0].tolist()
        stock_values.append(stock_values[0]-stock_values[1])
    else:
        stock_values = [pd.NA, pd.NA, pd.NA]
    return stock_values

Para entrenar la inteligencia articial, se debe tener un promedio de las puntuaciones de setnimiento, ya que al tener varias noticas para un día, es posible que se den múltiples interpretaciones si no se agrupan.

In [69]:
news_gb = news.groupby([news['affected_day'].dt.date, news['ticker']]).agg({
    'polarity_title': 'mean',
    'subjectivity_title': 'mean',
    'polarity_description': 'mean',
    'subjectivity_description': 'mean',
})

news_gb = news_gb.reset_index()
news_gb.tail()

,affected_day,ticker,polarity_title,subjectivity_title,polarity_description,subjectivity_description
469,2023-05-19,NVDA,0.105273,0.302131,0.059063,0.416841
470,2023-05-19,PYPL,0.203125,0.228125,0.468750,0.487500
471,2023-05-19,TSLA,0.083084,0.233050,0.092471,0.376658
472,2023-05-19,TTD,0.000000,0.000000,0.000000,0.000000
473,2023-05-19,ZG,0.000000,0.000000,0.000000,0.000000


In [70]:
news_gb[
    ['Open', 'Close', 'Open-Close']
] = news_gb.apply(lambda row: pd.Series(add_stock_results(row)), axis=1)
news_gb['target'] = np.where(
    news_gb['Open-Close'] > 0, 1, 0
)
test_output_path = os.path.join(final_results_path, "test.csv")
news_gb.to_csv(test_output_path, index=False)
news_gb.tail()

,affected_day,ticker,polarity_title,subjectivity_title,polarity_description,subjectivity_description,Open,Close,Open-Close,target
469,2023-05-19,NVDA,0.105273,0.302131,0.059063,0.416841,304.100006,312.640015,-8.540009,0
470,2023-05-19,PYPL,0.203125,0.228125,0.468750,0.487500,61.900002,60.919998,0.980003,1
471,2023-05-19,TSLA,0.083084,0.233050,0.092471,0.376658,177.169998,180.139999,-2.970001,0
472,2023-05-19,TTD,0.000000,0.000000,0.000000,0.000000,67.589996,66.82,0.769997,1
473,2023-05-19,ZG,0.000000,0.000000,0.000000,0.000000,44.529999,44.52,0.009998,1


In [71]:
news[
    ['Open', 'Close', 'Open-Close']
] = news.apply(lambda row: pd.Series(add_stock_results(row)), axis=1)
news_output_path = os.path.join(final_results_path, "news_complete.csv")
news.to_csv(news_output_path, index=False)
news.tail()

,ticker,title,description,pub_date,link,author,polarity_title,subjectivity_title,polarity_description,subjectivity_description,date,affected_day,Open,Close,Open-Close
1795,YELP,Companies announce abortion benefits as trigge...,Companies announce abortion benefits as trigge...,"Mon, 27 Jun 2022 14:46:00 GMT",https://finance.yahoo.com/video/companies-anno...,Yahoo Finance Video,0.0,0.0,0.0,0.0,2022-06-27 14:46:00,2022-06-27 14:46:00,30.52,29.58,0.940001
1796,YELP,Yelp closing D.C. office in commitment to remo...,Yelp closing D.C. office in commitment to remo...,"Mon, 27 Jun 2022 16:49:00 GMT",https://www.bizjournals.com/washington/news/20...,American City Business Journals,-0.1,0.2,-0.1,0.2,2022-06-27 16:49:00,2022-06-27 16:49:00,30.52,29.58,0.940001
1797,YELP,"Yelp (YELP) Embraces Remote Work Culture, Clos...","Yelp (YELP) Embraces Remote Work Culture, Clos...","Mon, 27 Jun 2022 14:06:00 GMT",https://finance.yahoo.com/news/yelp-yelp-embra...,Zacks,-0.1,0.2,-0.1,0.2,2022-06-27 14:06:00,2022-06-27 14:06:00,30.52,29.58,0.940001
1798,YELP,Yelp closes 3 U.S. offices as it shifts to rem...,Yelp closes 3 U.S. offices as it shifts to rem...,"Fri, 24 Jun 2022 21:09:00 GMT",https://finance.yahoo.com/video/yelp-closes-3-...,Yahoo Finance Video,-0.1,0.2,-0.1,0.2,2022-06-24 21:09:00,2022-06-25 10:30:00,<NA>,<NA>,<NA>
1799,YELP,Here's how Silicon Valley's companies are resp...,Here's how Silicon Valley's companies are resp...,"Fri, 24 Jun 2022 23:24:00 GMT",https://www.bizjournals.com/sanjose/news/2022/...,American City Business Journals,0.0,0.0,0.0,0.0,2022-06-24 23:24:00,2022-06-25 10:30:00,<NA>,<NA>,<NA>
